# Optimisation in Machine Learning


## Lab Assignment - 3

    Ayush Abrol B20AI052

---

In [2]:
import pandas as pd
import numpy as np
import cvxopt as cp
import cvxpy as cvx
import matplotlib.pyplot as plt
import yfinance as yf

### Looking Daily Close Price Historical Data from 30 August, 2021 to 30 August, 2022 for any 10 stocks.


Stocks selected: GOOG, AAPL, MS, AMZN, UBER, TSLA, NVDA, INTC, AMD, BABA

In [3]:
data = yf.download("GOOG AAPL MS AMZN UBER TSLA NVDA INTC AMD BABA", start="2021-08-30", end="2022-08-30")

[*********************100%***********************]  10 of 10 completed


In [4]:
data.head()

Adj Close                                                  \
                  AAPL         AMD        AMZN        BABA        GOOG   
Date                                                                     
2021-08-30  152.266769  111.320000  171.078506  162.289993  145.469498   
2021-08-31  150.983963  110.720001  173.539505  166.990005  145.462006   
2021-09-01  151.660141  109.989998  173.949997  173.279999  145.841995   
2021-09-02  152.793808  109.199997  173.156006  172.000000  144.218994   
2021-09-03  153.440186  109.919998  173.902496  170.300003  144.774994   

                                                                      ...  \
                 INTC          MS        NVDA        TSLA       UBER  ...   
Date                                                                  ...   
2021-08-30  52.218407  100.829178  226.725662  243.636673  39.590000  ...   
2021-08-31  52.334576  101.158539  223.737183  245.240005  39.139999  ...   
2021-09-01  51.957020  101.042290  224.296890  244.696671  40.619999  ...   
2021-09-02  52.015106  101.545998  223.847122  244.130005  41.400002  ...   
2021-09-03  51.802128  100.984169  228.314850  244.523331  40.320000  ...   

              Volume                                                    \
                AAPL       AMD      AMZN      BABA      GOOG      INTC   
Date                                                                     
2021-08-30  90956700  56130500  63844000  23769400  16916000  14581300   
2021-08-31  86453100  49339000  87128000  24629100  26756000  22350100   
2021-09-01  80313700  38579600  72598000  29550200  15824000  18615000   
2021-09-02  71115500  40090500  58474000  29302400  21844000  15869500   
2021-09-03  57808700  42605800  51514000  16423500  19104000  13456200   

                                                   
                 MS      NVDA      TSLA      UBER  
Date                                               
2021-08-30  6459700  26258000  55812600  29516300  
2021-08-31  7215100  25985000  62566200  26396000  
2021-09-01  5762400  20176700  39612900  27788600  
2021-09-02  5708400  18783400  38331900  20645400  
2021-09-03  7195400  28053200  45738300  15444900  

[5 rows x 60 columns]

In [5]:
df = data['Adj Close']

In [6]:
df.head()

,AAPL,AMD,AMZN,BABA,GOOG,INTC,MS,NVDA,TSLA,UBER
Date,,,,,,,,,,
2021-08-30,152.266769,111.320000,171.078506,162.289993,145.469498,52.218407,100.829178,226.725662,243.636673,39.590000
2021-08-31,150.983963,110.720001,173.539505,166.990005,145.462006,52.334576,101.158539,223.737183,245.240005,39.139999
2021-09-01,151.660141,109.989998,173.949997,173.279999,145.841995,51.957020,101.042290,224.296890,244.696671,40.619999
2021-09-02,152.793808,109.199997,173.156006,172.000000,144.218994,52.015106,101.545998,223.847122,244.130005,41.400002
2021-09-03,153.440186,109.919998,173.902496,170.300003,144.774994,51.802128,100.984169,228.314850,244.523331,40.320000


In [7]:
B = df.values
B.shape

(252, 10)

In [8]:
df.isnull().sum()

AAPL    0
AMD     0
AMZN    0
BABA    0
GOOG    0
INTC    0
MS      0
NVDA    0
TSLA    0
UBER    0
dtype: int64

In [9]:
Q = np.cov(B.T)      # Creating a covariance matrix

In [10]:
Q

array([[ 157.74375299,  125.09337764,  121.01897773,  -74.42080181,
          64.39093768,   13.44376116,   40.54330767,  335.77142599,
         391.54863568,   23.52915452],
       [ 125.09337764,  439.04994113,  377.71547605,  250.72765389,
         229.59182499,   75.662261  ,  130.22328823,  969.71227629,
         742.57837965,  121.74834565],
       [ 121.01897773,  377.71547605,  540.13457818,  420.63647601,
         296.01481426,  103.04531458,  168.8456825 ,  943.04805884,
         722.52832314,  173.67593213],
       [ -74.42080181,  250.72765389,  420.63647601,  711.69523388,
         253.5054141 ,   98.30137056,  156.78364586,  557.42907145,
         267.38458803,  169.15405922],
       [  64.39093768,  229.59182499,  296.01481426,  253.5054141 ,
         178.65881525,   63.05593942,   96.33548701,  570.16909761,
         406.11457314,   97.11347355],
       [  13.44376116,   75.662261  ,  103.04531458,   98.30137056,
          63.05593942,   32.01402607,   36.17833611,  191

In [11]:
c = np.array([[0] for i in range (10)])
m = -np.log(np.array([B[-1][i]/B[0][i] for i in range (10)]))
m

array([-0.05812782,  0.22951941,  0.27620485,  0.51880609,  0.27639995,
        0.46074737,  0.16666735,  0.36108246, -0.15617936,  0.32028669])

In [12]:
R = 52 #B20AI052
val = R/1000

In [13]:
A = np.column_stack((m, np.negative(np.identity(10, dtype = 'int'))))
b = np.array([-val]+[0]*10)
Aeq = np.array([[1]*10])
beq = np.array([[1]])
A.shape

(10, 11)

In [14]:
A = A.T
A.shape

(11, 10)

In [15]:
sol = cp.solvers.qp(cp.matrix(Q, tc='d'), cp.matrix(c, tc='d'), cp.matrix(A, tc='d'), cp.matrix(b, tc='d'), cp.matrix(Aeq, tc='d'), cp.matrix(beq, tc='d'))
print(sol['x'])

     pcost       dcost       gap    pres   dres
 0:  3.7802e+00  3.5056e+00  2e+01  5e+00  4e+00
 1:  3.8295e+00  4.4471e+00  6e+00  1e+00  1e+00
 2:  1.1823e+01  1.9949e+01  2e+01  1e+00  9e-01
 3:  1.7238e+01  3.3746e+01  2e+01  6e-01  5e-01
 4:  2.1496e+01  3.8895e+01  2e+01  4e-01  3e-01
 5:  2.9321e+01  5.0953e+01  1e+01  2e-01  2e-01
 6:  4.0262e+01  6.3981e+01  1e+01  1e-01  1e-01
 7:  7.5309e+01  7.5476e+01  7e+00  2e-02  1e-02
 8:  7.4515e+01  7.4930e+01  5e+00  9e-03  8e-03
 9:  7.5854e+01  7.5598e+01  4e-01  2e-04  2e-04
10:  7.5735e+01  7.5732e+01  5e-03  3e-06  2e-06
11:  7.5733e+01  7.5733e+01  5e-05  3e-08  2e-08
Optimal solution found.
[ 9.73e-01]
[ 6.64e-08]
[ 2.17e-08]
[ 3.31e-09]
[ 4.97e-08]
[ 2.55e-08]
[ 2.73e-02]
[ 3.32e-09]
[-2.36e-09]
[ 7.52e-08]



In [16]:
print("After rounding upto 5 decimal places: ")
for i in range(10):
    print(round(sol['x'][i], 5))

print("The minimum variance portfolio is: ", sol['primal objective'])

After rounding upto 5 decimal places: 
0.97274
0.0
0.0
0.0
0.0
0.0
0.02726
0.0
-0.0
0.0
The minimum variance portfolio is:  75.73311853662078


---